# Titanic - Exploratory Data Analysis - 05

In [1]:
# Import libraries
import os
import sys

import scipy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

import warnings

sys.path.append('Titanic\\Code\\DataPrep')
from helpers import *

warnings.filterwarnings('ignore')
np.random.seed(17)
%matplotlib inline

## 1. Read and summarize the data

### 1.1 Read infer column types

In [3]:
df_train = get_titanic_data(train_or_test='train')

#### Data Dictionary

  * Categorical
    * Survival (Target variable)
        * 0 = No
        * 1 = Yes
    * Pclass: Ticket class - proxy for socio-economic status
        * 1 = 1st
        * 2 = 2nd
        * 3 = 3rd
    * Sex
    * Embarked: Port of Embarkation
        * C = Cherbourg
        * Q = Queenstown
        * S = Southampton
  * Numerical
    * Age: Age in years - fractional if less than 1
        * If the age is estimated, it is in the form of xx.5
    * SibSp: \# of siblings / spouses aboard the Titanic
        * Sibling = brother, sister, stepbrother, stepsister
        * Spouse = husband, wife (mistresses and fiances were ignored)
    * Parch: \# of parents / children aboard the Titanic
        * Parent = mother, father
        * Child = daughter, son, stepdaughter, stepson
        * Some children travelled only with a nanny, therefore parch=0 for them
    * Fare: Passenger fare

### 1.2 Print the first n rows of data

In [5]:
df_train.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,8.4583,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,51.8625,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,21.0750,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,11.1333,S


### 1.3 Print the dimensions, column names, and types of the data